In [1]:
from exp8_feature_extraction import get_balanced_dataset
from scripts.cross_validate import run_cross_validate

import numpy as np
import gensim

import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
all_reviews = get_balanced_dataset()

In [3]:
len(all_reviews)

160933

In [4]:
reviews_contents = [x.review_content for x in all_reviews]
labels = [1 if x.label else 0 for x in all_reviews]

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(reviews_contents)
input_features = np.array(pad_sequences(tokenizer.texts_to_sequences(reviews_contents)))

In [ ]:
corpus_words = tokenizer.word_index
corpus_vocab_size = len(corpus_words)+1

In [ ]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format("../../data/GoogleNews-vectors-negative300.bin",
                                                               binary=True)

In [ ]:
embedding_length = word_vectors.vector_size
embedding_matrix = np.zeros((corpus_vocab_size, embedding_length))
for word, index in corpus_words.items():
  if word in word_vectors.vocab:
    embedding_matrix[index] = np.array(word_vectors[word], dtype=np.float32)

In [ ]:
max_review_len = 0
for input_feature in input_features:
    max_review_len = max(max_review_len, len(input_feature))
print(max_review_len)

997


In [ ]:
corpus_vocab_size

103804

In [ ]:
len([x.label for x in all_reviews if x.label == 0])

80467

In [ ]:
def get_ff_wv_model():
  model_ff_wv = tf.keras.Sequential([
      tf.keras.layers.Embedding(corpus_vocab_size, embedding_length, weights=[embedding_matrix], trainable=False,
          input_length=max_review_len), # input length supposed to be length of review
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(16, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
      tf.keras.layers.Dropout(0.25),
      tf.keras.layers.Dense(8, activation=tf.keras.activations.relu, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
      tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)
  ])
  model_ff_wv.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model_ff_wv

In [ ]:
ff_wv_scores = run_cross_validate(get_ff_wv_model, input_features, labels, cv=10, epochs=24)
print(ff_wv_scores)

Fitting with:  (144839, 997) labels (144839,)
Train on 101387 samples, validate on 43452 samples
Epoch 1/24
 34496/101387 [=========>....................] - ETA: 3:16 - loss: 0.7998 - acc: 0.5741

In [16]:
output.show()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
ff_wv_scores

{'accuracies': [0.608052690495493,
  0.5452963837454952,
  0.5460420032236107,
  0.5683484528099393,
  0.5538088728755812,
  0.6043245930604733,
  0.6024358416258463,
  0.5802261993833481,
  0.6019139945166282,
  0.5000000000148159]}